In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from math import cos, sin, exp, sqrt, pi, e
from datetime import datetime
from pandas import DataFrame

mypalette = sns.color_palette(['#162f4d']) # https://www.hsluv.org/
sns.set_palette(mypalette)

## Read the dataset

In [ ]:
Tamb_input_path = os.path.join(".", "data", "Ospitaletto", "processed", "Tamb_input.xls")
Tamb_hourly_path = os.path.join(".", "data", "Ospitaletto", "processed", "Tamb_hourly.xls")

In [ ]:
temp=pd.read_excel(Tamb_input_path ,squeeze=True)
Treal=pd.read_excel(Tamb_hourly_path, squeeze=True)

Tamb_fit=pd.Series(temp)
Tamb=pd.Series(Treal)

Tamb.plot()
Tamb_fit.plot(color='red')
print('Tamb_min:',Tamb_fit.min(),'Tamb_max',Tamb_fit.max())


In [ ]:
date_rng = pd.date_range(start='1/1/2019 00:00:00', end='31/12/2019 00:00:00', freq='H')
df = pd.DataFrame(date_rng, columns=['date'])

df['Tamb_fit'] = Tamb_fit
df['Tamb'] = Tamb


df.head(12)
#df[df.index.month==12]



# Data manipulation-processing

In [ ]:
Tamb_hourly=pd.DataFrame(df, columns = ['date','Tamb','Tamb_fit']) #Ambient temperature on an hourly resolutionT


In [ ]:
Tamb_hourly['date'] =pd.to_datetime(Tamb_hourly['date'], infer_datetime_format=True) #Estoy asociando el index de tiempo a la columna Data-Ora
Tamb_hourly=Tamb_hourly.set_index('date')

In [ ]:
Tamb_hourly.head()

In [ ]:
Tamb_hourly.info()

In [ ]:
Tamb_hourly.index

In [ ]:
# Tamb_hourly.reset_index(inplace=True)

In [ ]:
Tamb_hourly['Month'] = Tamb_hourly.index.month


In [ ]:
Tamb_hourly

In [ ]:
Tmin_i = float(input("Enter min setpoint temperature: "))
Tmax_i = float(input("Enter max setpoint temperature: "))

In [ ]:
def climatic_curve(Tamb_h):
    Tmin_o = 2.38 #minimum outdoor T threshold in which the space heating system turns on
    Tmax_o = 7.25 #maximum outdoor T threshold in which the space heating system turns off

    if Tamb_h <= Tmin_o:   
        Tsh = Tmax_i
    elif Tamb_h >= Tmax_o:
        Tsh = Tmin_i
    else:
        m = (Tmax_i-Tmin_i)/(Tmin_o-Tmax_o)
        b = -m*Tmin_o+Tmax_i
        Tsh = m*Tamb_h+b
    return Tsh
 

In [ ]:
Tamb_hourly['Tsh'] = Tamb_hourly['Tamb'].apply(climatic_curve)
#Tamb_hourly['Tsh'] = Tamb['Temp'].apply(lambda x: climatic_curve(x))
Tamb_hourly

In [ ]:
Tdhw = float(input("Enter domestic hot water temperature: "))

Tamb_hourly["Tdhw"] = Tdhw
Tamb_hourly

In [ ]:
Tamb_hourly.head(12)


In [ ]:
november_data = Tamb_hourly[Tamb_hourly.index.month == 11 ]
november_data[["Tsh","Tdhw"]].plot(figsize=(10,6), color=["#4e97a2",'#82272e'])

plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
Tamb_hourly[["Tsh","Tdhw"]].dtypes

In [ ]:
def Tuser(col):
    Tsh=col["Tsh"]
    Tdhw=col["Tdhw"]
    fecha=col.name
    #Las proporciones dependen de la localización
    if fecha.month==1:
        resultado= 0.98*Tsh+0.02*Tdhw
       
    elif fecha.month==2:
        resultado=0.96*Tsh+0.04*Tdhw
    
    elif fecha.month==3:
        resultado=0.94*Tsh+0.06*Tdhw
    
    elif fecha.month==4:
        resultado=0.90*Tsh+0.1*Tdhw
    
    elif fecha.month==5:
        resultado=0.75*Tsh+0.25*Tdhw
    
    elif fecha.month==6:
        resultado=0.25*Tsh+0.75*Tdhw
    
    elif fecha.month==7:
        resultado=0*Tsh+1.0*Tdhw
    
    elif fecha.month==8:
        resultado=0*Tsh+1.0*Tdhw
    
    elif fecha.month==9:
        resultado=0.1*Tsh+0.9*Tdhw
    
    elif fecha.month==10:
        resultado=0.80*Tsh+0.2*Tdhw
    
    elif fecha.month==11:
        resultado=0.95*Tsh+0.05*Tdhw
    
    elif fecha.month==12:
        resultado=0.97*Tsh+0.03*Tdhw
        
    return resultado

In [ ]:
Tamb_hourly["Tuser"] = Tamb_hourly.apply(lambda x: Tuser(x), axis=1)

In [ ]:
Tamb_hourly["Tuser"].plot(figsize=(10,6))

In [ ]:
november_data = Tamb_hourly[Tamb_hourly.index.month == 11 ]
november_data["Tuser"].plot(figsize=(10,6))

# Network temperature 

In [ ]:
zz=1               #Depth [m]
alpha=0.06048      #Ground thermal diffusivity, Banks [m^2/day]
alpha_sec=7e-7 #Ground thermal diffusivity, Banks [m^2/s]
t_sec=365*24*3600
d_shift_max = Tamb_hourly['Tamb_fit'].idxmax(axis=0)
dd_max=(Tamb_hourly.index.get_loc(d_shift_max))/24

Tg_und = Tamb.mean() #T_ave_fit #Undisturbed ground temperature
DT_y= 11.23 #abs(DT_y_fit)



In [ ]:
def ground_temperature_hour(t,zz):
    #t is time in hours, but the calculation is done is seconds
    T_banks= Tg_und+DT_y*exp(-zz*sqrt(pi/(alpha_sec*t_sec)))*cos(2*pi/t_sec*(t-dd_max*24)*3600-zz*sqrt(pi/(alpha_sec*t_sec)))
    
    return pd.Series(T_banks)

In [ ]:
Tamb_hourly = Tamb_hourly.reset_index(drop=False)
Tamb_hourly

In [ ]:
Tamb_hourly.index

In [ ]:
Tamb_hourly["Tground"] = Tamb_hourly.apply(lambda fila: ground_temperature_hour(fila.name + 1,1), axis=1) # x + 1 because it starts on 0.
Tamb_hourly

In [ ]:
Tamb_hourly[["Tground"]].plot()

In [ ]:
depth_aquifer= float(input("""Aquifer temperature depth:
"""))

In [ ]:
# fila.name because that's the index (numeric range from 0 to 8737)
# fila.name + 1 because the range starts in 0 and the function assumes it begins with 1.
Tamb_hourly["Taq"] = Tamb_hourly.apply(lambda fila: ground_temperature_hour(fila.name + 1, depth_aquifer), axis=1)
Tamb_hourly

In [ ]:
Tamb_hourly[["Taq"]].plot()

In [ ]:
Tamb_hourly[["Taq", "Tground"]].plot()

In [ ]:
Tamb_hourly = Tamb_hourly.set_index("date")
Tamb_hourly

## Load Sources working hours

The file must be a boolean matrix $W$ of $24 x 7$. The rows represent the hours in a day, and the columns represent the days in the week. For all $i \in \{0,...,23\}$ and $j \in \{0,..., 6\}$ we have that $w_{i,j} \in \{0, 1\}$, where $w_{i,j} = 0$ means that the source doesn't produce energy at that hour $i$ on that day $j$. Similarly, $w_{i,j} = 1$ means that the source produces energy at that hour $i$ on that day $j$.

In [ ]:
s1_schedule_path = os.path.join(".", "data", "private", "s1_source_fake_schedule.xlsx")
s1_schedule_path

In [ ]:
s2_schedule_path = os.path.join(".", "data", "private", "s2_source_fake_schedule.xlsx")
s2_schedule_path

In [ ]:
# Time repr
s1_schedule = pd.read_excel(s1_schedule_path, index_col="Time")
s1_schedule

In [ ]:
s1_schedule = s1_schedule.to_numpy()
s1_schedule

In [ ]:
# Tuesdays are closed at 6 but open are 7
s1_schedule[6,1], s1_schedule[7,1]

In [ ]:
# Time repr
s2_schedule = pd.read_excel(s2_schedule_path, index_col="Time")
s2_schedule

In [ ]:
s2_schedule = s2_schedule.to_numpy()
s2_schedule

In [ ]:
# Tuesdays are open at 16 but closed are 17
s2_schedule[15,1], s2_schedule[16,1]

In [ ]:
Tamb_hourly

## Calculate Sources Temperature based on Working Hours

In [ ]:
def get_temp_by_working_hours(temp_src, datetime, schedule):
    day_of_week = datetime.dayofweek
    hour_of_day = datetime.hour
    return temp_src * (schedule[hour_of_day, day_of_week])

In [ ]:
Ts1 = float(input("""Temperature of source 1: 
"""))

Ts2 = float(input("""Temperature of source 2: 
"""))


In [ ]:
Ts1 = Tamb_hourly.apply(lambda fila: get_temp_by_working_hours(Ts1, fila.name, s1_schedule), axis=1)
Ts1.head(50)

In [ ]:
Ts2 = Tamb_hourly.apply(lambda fila: get_temp_by_working_hours(Ts2, fila.name, s2_schedule), axis=1)
Ts2.head(50)

In [ ]:
Tamb_hourly["Ts1"] = Ts1
Tamb_hourly["Ts2"] = Ts2

Tamb_hourly

In [ ]:
def calculate_tnet(temp_s1, temp_s2, temp_aq):
    if temp_s1 == 0.0 and temp_s2 == 0.0:
        return temp_aq
    elif temp_s1 == 0.0:
        return temp_s2
    elif temp_s2 == 0.0:
        return temp_s1
    else:
        return np.mean([temp_s1, temp_s2])

In [ ]:
Tamb_hourly["Tnet"] = Tamb_hourly.apply(lambda fila: calculate_tnet(fila["Ts1"], fila["Ts2"], fila["Taq"]), axis=1)
Tamb_hourly.head(50)

# Coefficient of performance (COP)

In [ ]:
DT_evap =float(input("""Temperature difference among the supply and return pipes: 
#"""))

DT_hx=2.5
n_HP = 0.53
Te_o =Tamb_hourly['Tnet']-DT_evap-DT_hx
Tc_o =Tamb_hourly['Tuser']

Tamb_hourly['COP']= 1-n_HP+n_HP*(Tc_o+273.15)/(Tc_o+DT_hx-Te_o)

In [ ]:
Tamb_hourly[['COP']].plot(figsize=(10,6))
#Tc_o.plot()